In [1]:
# PDF Reader
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader('kpis.pdf')
pdf_documents = loader.load()


In [2]:
# Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=300
)

# Split the documents into chunks
documents = text_splitter.split_documents(pdf_documents)


In [8]:
# Vector Embedding and Vector Store
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize the embedding model
embedding_model = OllamaEmbeddings(model="llama3.2:3b")  

# Load the persisted Chroma vector store
db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

db

In [7]:
# Query the database
query = "Why is this indicator important?"

# Perform a similarity search
response = db.similarity_search(query, k=3)

# Clean the result content (remove unwanted tab characters and extra spaces)
cleaned_response = response[0].page_content.replace("\t", " ").strip()
cleaned_response = " ".join(cleaned_response.split())

print(cleaned_response)


actually completed at any given point in time. In addition to assessing progress to date, EV allows companies to project what the likely costs of the complete project will be, assuming that performance levels remain as they have been to date. How do I measure it?


In [10]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

llm

C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_10032\1732197747.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


Ollama()

## Retriever And Chain With Langchain

In [11]:

## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
<context>
{context}
</context>
Question: {input}""")

In [12]:
# Chain instruction 
# Create Stuff Documnt Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [13]:
# Set up the retriever
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001A2947898E0>, search_kwargs={})

In [14]:
# Retriever Chain

from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001A2947898E0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context. \nThink step by step before providing a detailed answer. \n<

In [15]:
response = retrieval_chain.invoke({"input":"Why is this indicator important?"})

In [16]:
response['answer']

"The indicator of carbon footprint is important for several reasons:\n\n1. Regulatory compliance: With ever-increasing legislation and regulations around environmental sustainability, measuring carbon footprint can help organizations comply with these requirements and avoid penalties or fines.\n2. Reputation management: In today's socially conscious world, not performing well against carbon-footprint measures can have devastating effects on an organization's reputation, which can impact profits and share price.\n3. Cost savings: Measuring and managing carbon footprint can lead to cost savings through reducing energy consumption, lowering emissions, and optimizing resource usage.\n4. Long-term sustainability: By measuring and reducing their carbon footprint, organizations can contribute to long-term environmental sustainability and reduce their impact on the planet.\n5. Stakeholder expectations: Investors, customers, and employees are increasingly expecting organizations to prioritize e